In [ ]:
#| default_exp train

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch.distributed as dist, torch, os, tiktoken, numpy as np, time
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP

from scratchRLHF.models.gpt2 import *

In [ ]:
#| export
def load_tokens(filename):
    npt = np.load(filename).astype(np.int64)
    ptt = torch.tensor(npt, dtype=torch.long)
    return ptt

class DataLoaderLite:

    def __init__(self, B, T, process_rank, num_processes, split):
        self.B, self.T = B, T
        self.process_rank = process_rank
        self.num_processes = num_processes
        assert split in {'train', 'val'}

        # get shard files
        data_root = "/scratch/scai/phd/aiz218323/datasets/deepreviewer/cache/"
        shards = os.listdir(data_root)
        shards = [s for s in shards if split in s]
        shards = sorted(shards)
        shards = [os.path.join(data_root, s) for s in shards]
        self.shards = shards
        assert len(shards) > 0, f"no shard found for split {split}"
        if master_process:
            print(f'found {len(shards)} shards for split {split}')

        # state, init at shard zero
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T * self.process_rank

    def reset(self):
        self.current_shard = 0
        self.tokens = load_tokens(self.shards[self.current_shard])
        self.current_position = self.B * self.T * self.process_rank

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position:self.current_position+(B*T)+1]
        x = buf[:-1].view(B, T)
        y = buf[1:].view(B, T)

        self.current_position += (B * T * self.num_processes)
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_shard = (self.current_shard + 1) % len(self.shards)
            self.tokens = load_tokens(self.shards[self.current_shard])
            self.current_position = self.B * self.T * self.process_rank
        return x, y
        

In [ ]:
#| export
ddp = int(os.environ.get('RANK', -1)) != -1

if ddp:
    assert torch.cuda.is_available(), "CUDA is required for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0
else:
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = 'mps'
    print(f'Using device : {device}')

device_type = "cuda" if device.startswith("cuda") else "cpu"

Using device : cpu


In [ ]:
#| export
total_batch_size = 524288
B = 8
T = 1024

assert total_batch_size % (B * T * ddp_world_size) == 0, "make sure total_batch_size is divisible by B * T * ddp_world_size"

grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
if master_process:
    print(f'total desired batch size: {total_batch_size}')
    print(f'=> calculated gradient accumulation steps: {grad_accum_steps}')
    

total desired batch size: 524288
=> calculated gradient accumulation steps: 64


In [ ]:
#| export
model_path = '/home/scai/phd/aiz218323/scratch/outputs/scratchRLHF/gpt2.pth'

model = GPT.from_pretrained('gpt2', model_path)
model = model.to(device)
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model

loading weights from pretrained gpt: gpt2


In [ ]:
#| export
model = torch.compile(model)

In [ ]:
#| export
train_dataloader = DataLoaderLite(B=8, T=1024, process_rank=ddp_rank, num_processes=ddp_world_size, split='train')
val_dataloader = DataLoaderLite(B=8, T=1024, process_rank=ddp_rank, num_processes=ddp_world_size, split='val')

found 4 shards for split train
found 1 shards for split val


In [ ]:
#| export
max_lr = 6e-4
min_lr = max_lr * 0.1

# warmup_steps = 10
# max_steps = 50

warmup_steps = 715
max_steps = 19073

save_steps = 1000
model_dir = "/home/scai/phd/aiz218323/scratch/outputs/scratchRLHF/models"
os.makedirs(model_dir, exist_ok=True)

def get_lr(it):
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps

    if it > max_steps:
        return min_lr

    decay_ratio = (it - warmup_steps)/(max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)
    

In [ ]:
#| export
log_dir = "/home/scai/phd/aiz218323/scratch/outputs/scratchRLHF/log"
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'log.txt')
with open(log_file, 'w') as f:
    pass
    

In [ ]:
#| export
optimizer = raw_model.configure_optimizers(weight_decay=0.1, learning_rate=6e-4, device=device)

for step in range(max_steps):
    t0 = time.time()
    last_step = (step == max_steps - 1)

    if step % 100 == 0 or last_step:
        model.eval()
        val_dataloader.reset()
        with torch.no_grad():
            val_loss_accum = 0.0
            val_loss_steps = 20
            for _ in range(val_loss_steps):
                x, y = val_dataloader.next_batch()
                x, y = x.to(device), y.to(device)
                with torch.autocast(device_type=device_type, dtype=torch.bfloat16):
                    logits, loss = model(x, y)
                loss = loss / val_loss_steps
                val_loss_accum += loss.detach()
        if ddp:
            dist.all_reduce(val_loss_accum, op=dist.ReduceOp.AVG)
        if master_process:
            print(f'validation loss: {val_loss_accum.item():.4f}')
            with open(log_file, 'w') as f:
                f.write(f'{step} val {val_loss_accum.item():.4f}\n')
                
        if step % 1000 == 0 or last_step:
            raw_model.save(f"{model_dir}/gpt2_{step:05d}.pth")

    model.train()
    optimizer.zero_grad()
    
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        x, y = train_dataloader.next_batch()
        x, y = x.to(device), y.to(device)
        logits, loss = model(x, y)
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)
        loss.backward()
    if ddp:
        dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    optimizer.step()
    torch.cuda.synchronize()
    
    t1 = time.time()
    dt = (t1 - t0) * 1000
    tokens_per_sec = (train_dataloader.B * train_dataloader.T * grad_accum_steps * ddp_world_size)/(t1 - t0)
    print(f"Step {step} | loss: {loss_accum.item()} | lr: {lr:.4e} norm: {norm:.4f} | dt: {dt:.2f}ms | tok/sec: {tokens_per_sec:.2f}")
    

In [ ]:
#| export
if ddp:
    destroy_process_group()